<a href="https://colab.research.google.com/github/theharshithh/onfinace-assignment/blob/main/with_bge_embeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
!pip -q install langchain openai tiktoken chromadb langchain_openai

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_openai import OpenAI depricatred
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings


def read_doc(dir):
    file_loader = PyPDFDirectoryLoader(dir)
    docs = file_loader.load()
    return docs

loading data to /dataset dir

In [ ]:
!pip install pypdf sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
primary_data = read_doc('dataset/')
primary_data

creating chunks for the `primary_data`

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
spilit_texts = text_splitter.split_documents(primary_data)

## create the embeddings using hf ede by baai

In [ ]:
#hugging face bge embeddings
model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

change of embeddings from `OpenAIEmbeddings()` to bge-embeddings

In [ ]:
persist_directory = 'db-on'

#openai embeddings -> converts texts to vecs
# embedding = OpenAIEmbeddings()
embedding = model_norm
#load the spilt chunk vectors into the vectordb
vectordb = Chroma.from_documents(documents=spilit_texts, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:

user_query = 'Considering 2 wheeler demand, who seems to be the segment winner?'

In [ ]:
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents(user_query)
#getting the relevant docs for the user query
docs

[Document(page_content="UV's  29,171  28,099  3.8 28,796  1.3 3,63,332  2,27,506  59.7  \nHero MotoCorp  4,34,165  4,50,170  -3.6 3,94,460  10.1  52,43,369  49,44,148  6.1 \nBajaj Auto  2,81,922  2,97,188  -5.1 2,80,226  0.6 39,16,077  43,08,433  -9.1 \n  Two -Wheelers  2,33,703  2,56,324  -8.8 2,35,356  -0.7 34,27,405  38,36,856  -10.7  \n  Three -Wheelers  48,219  40,864  18.0  44,870  7.5 4,88,672  4,71,577  3.6 \nDomestic  1,64,775  1,26,752  30.0  1,53,291  7.5 20,82,735  18,01,807  15.6  \nExports  1,17,146  1,70,436  -31.3  1,26,935  -7.7 18,33,341  25,06,626  -26.9  \nAshok Leyland  20,894  20,123  3.8 18,571  12.5  1,89,173  1,28,326  47.4  \nM&HCV  14,197  13,990  1.5 12,668  12.1  1,21,533  73,885  64.5  \nLCV  6,697  6,133  9.2 5,903  13.5  67,640  54,441  24 \nTVS Motor  2,92,177  3,07,954  -5.1 2,76,150  5.8 36,57,093  33,09,578  10.5  \nMotorcycles  1,37,970  1,60,522  -14.0  1,26,243  9.3 17,29,976  17,31,459  -0.1 \nScooters  1,05,785  94,747  11.6  1,04,825  0.9 12,97

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

### LLM chain

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-4-turbo", temperature=1.2 ), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [ ]:
def process_llm_response(llm_response):
    print("Decision:")
    print(llm_response['result'])

    print("\n Source Documents:")
    for source in llm_response["source_documents"]:
        print("- " + source.metadata['source'])

    print("\nRelevant Data:")
    for source in llm_response["source_documents"]:
        print(source.page_content)

In [ ]:
behaviour = '''Think like an expert finance consultant from a top tier firm. Do all the research and analysis for this document.'''
prompt_template = 'Keep the answer crisp. Give me as much numbers as you can. Every Statement of yours should be backed with data'
stop_prompt = f'Dont hallucinate, if u dont know the answer, tell you dont know'
query = f"Here is the user query : {user_query}. Remember this: {behaviour}. Give me the answers like this: {prompt_template}. Keep this in mind - {stop_prompt}"
llm_response = qa_chain(query)
# llm_response
process_llm_response(llm_response)

Decision:
Reviewing the sales metric data provided for various companies in the two-wheeler segment, Hero MotoCorp stands out despite a brief decline in performance. 

1. **Hero MotoCorp** observed significant overall sales volumes despite a year-on-year decline. They sold 434,165 units in the recent reported period compared to 450,170 units in the prior corresponding period, reflecting a decrease of roughly 3.6%. However, on a sequential basis, there was an impressive recovery (from 394,460 in a previous period), and they have achieved a cumulative growth of 6.1% year-over-year.

2. In comparison, **TVS Motor** reports sales of motorcycles at 137,970, witnessing a 14% decrease while scooters improved by 11.6% to 106,785 units. The negative growth in motorcycles might signal a concerning trend, albeit compensated by the growth in the scooter segment.

3. **Bajaj Auto**, in collaboration with exported sales figures, reported a year-on-year decrease by 5.1% in overall delivery.

Given th